## 1. Import Libraries

In [2]:
import pandas as pd
import sqlite3
from sqlalchemy import create_engine, text
from datetime import datetime
import os

print("✓ Libraries imported successfully")

✓ Libraries imported successfully


## 2. Configuration

In [3]:
# Database Configuration
DB_PATH = 'data/risk_profiling.db'
TABLE_NAME = 'risk_profiling_monthly_data'

# CSV File Path
CSV_FILEPATH = 'data/investor_risk_appetite_dataset.csv'

# Snapshot Date Configuration
# Option 1: Use last day of previous month (automatic)
def get_last_month_end_date():
    """Calculate the last day of the previous month."""
    today = datetime.now()
    first_day_current_month = today.replace(day=1)
    last_day_previous_month = first_day_current_month - pd.Timedelta(days=1)
    return last_day_previous_month.strftime('%Y-%m-%d')

# Option 2: Manual override (uncomment to use specific date)
# SNAPSHOT_DATE = '2025-11-30'

# Use automatic calculation
SNAPSHOT_DATE = get_last_month_end_date()

print(f"\n✓ Configuration:")
print(f"  CSV File: {CSV_FILEPATH}")
print(f"  Database: {DB_PATH}")
print(f"  Table: {TABLE_NAME}")
print(f"  Snapshot Date: {SNAPSHOT_DATE}")


✓ Configuration:
  CSV File: data/investor_risk_appetite_dataset.csv
  Database: data/risk_profiling.db
  Table: risk_profiling_monthly_data
  Snapshot Date: 2025-11-30


## 3. Load CSV Data

In [4]:
# Check if CSV file exists
if not os.path.exists(CSV_FILEPATH):
    raise FileNotFoundError(f"CSV file not found: {CSV_FILEPATH}")

# Read CSV file
print(f"\n{'='*100}")
print(f"LOADING CSV DATA")
print(f"{'='*100}")

print(f"\n✓ Reading CSV file: {CSV_FILEPATH}")
df = pd.read_csv(CSV_FILEPATH)

print(f"\n✓ CSV Data Loaded:")
print(f"  Total Records: {len(df):,}")
print(f"  Total Columns: {df.shape[1]}")
print(f"  Memory Usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Display column names
print(f"\n✓ Columns ({df.shape[1]}):")
for i, col in enumerate(df.columns, 1):
    print(f"  {i:3d}. {col}")

# Display data split distribution
if 'data_split' in df.columns:
    print(f"\n✓ Data Split Distribution:")
    split_counts = df['data_split'].value_counts()
    for split, count in split_counts.items():
        print(f"  {split:12s}: {count:>6,} ({count/len(df)*100:.1f}%)")

# Display first few rows
print(f"\n✓ Sample Data (first 3 rows):")
display(df.head(3))


LOADING CSV DATA

✓ Reading CSV file: data/investor_risk_appetite_dataset.csv

✓ CSV Data Loaded:
  Total Records: 125,000
  Total Columns: 111
  Memory Usage: 158.65 MB

✓ Columns (111):
    1. customer_id
    2. customer_segment
    3. age
    4. gender
    5. marital_status
    6. dependents
    7. education
    8. city_tier
    9. employment_status
   10. occupation_sector
   11. years_employed
   12. annual_income
   13. monthly_income
   14. monthly_expenses
   15. total_savings
   16. investment_portfolio_value
   17. liquid_cash
   18. total_debt
   19. home_loan_amount
   20. personal_loan
   21. credit_card_outstanding
   22. auto_loan
   23. monthly_credit_amount_1m
   24. monthly_debit_amount_1m
   25. avg_monthly_credit_6m
   26. avg_monthly_debit_6m
   27. debit_to_credit_ratio_1m
   28. debit_to_credit_ratio_6m
   29. debt_to_income_ratio
   30. savings_to_income_ratio
   31. investment_to_income_ratio
   32. credit_utilization
   33. net_worth
   34. relationship_vinta

,customer_id,customer_segment,age,gender,marital_status,dependents,education,city_tier,employment_status,occupation_sector,...,emergency_fund_adequacy,insurance_adequacy_ratio,credit_card_payment_history_score,loan_repayment_track_record,financial_planning_score,budgeting_discipline_score,retirement_planning_score,tax_planning_efficiency,data_split,risk_profile
0,100000,Mass Market,50,Male,Single,4,Graduate,Metro,Salaried,Other,...,0.33,23.63,54.3,14.1,80.2,7.5,98.7,77.2,Train,Conservative
1,100001,Mass Market,32,Male,Single,2,Professional,Metro,Salaried,IT,...,0.33,18.22,32.3,51.9,70.3,36.4,97.2,96.2,Train,Aggressive
2,100002,Emerging Affluent,58,Male,Married,0,Others,Tier 1,Salaried,Finance,...,0.50,14.29,0.9,10.1,66.4,0.5,16.1,54.9,Train,Aggressive


## 4. Add Snapshot Date Column

In [5]:
# Add snapshot_date column
df['snapshot_date'] = SNAPSHOT_DATE

print(f"\n✓ Added snapshot_date column: {SNAPSHOT_DATE}")
print(f"  Total columns now: {df.shape[1]}")
print(f"  New column added at position: {list(df.columns).index('snapshot_date') + 1}")


✓ Added snapshot_date column: 2025-11-30
  Total columns now: 112
  New column added at position: 112


## 5. Save to SQLite Database

In [6]:
print(f"\n{'='*100}")
print(f"SAVING DATA TO SQLITE DATABASE")
print(f"{'='*100}")

# Create directory if it doesn't exist
os.makedirs(os.path.dirname(DB_PATH), exist_ok=True)

# Create SQLite connection
engine = create_engine(f'sqlite:///{DB_PATH}')

print(f"\n✓ Database Connection:")
print(f"  Database: {DB_PATH}")
print(f"  Table: {TABLE_NAME}")

# Check if table exists and has data for this snapshot
table_exists = False
existing_records = 0

try:
    with engine.connect() as conn:
        result = conn.execute(text(f"SELECT name FROM sqlite_master WHERE type='table' AND name='{TABLE_NAME}'"))
        if result.fetchone():
            table_exists = True
            result = conn.execute(text(f"SELECT COUNT(*) FROM {TABLE_NAME} WHERE snapshot_date = '{SNAPSHOT_DATE}'"))
            existing_records = result.fetchone()[0]
except:
    pass

if existing_records > 0:
    print(f"\n⚠️  Warning: Found {existing_records:,} existing records for snapshot {SNAPSHOT_DATE}")
    print(f"  These will be deleted and replaced with new data")
    
    # Delete existing records for this snapshot
    with engine.connect() as conn:
        conn.execute(text(f"DELETE FROM {TABLE_NAME} WHERE snapshot_date = '{SNAPSHOT_DATE}'"))
        conn.commit()
    print(f"  ✓ Deleted {existing_records:,} existing records")

# Determine if_exists strategy
if_exists_strategy = 'append' if table_exists else 'replace'

print(f"\n✓ Saving data to database...")
print(f"  Strategy: {if_exists_strategy}")
print(f"  Records to insert: {len(df):,}")

# Save to database
df.to_sql(TABLE_NAME, engine, if_exists=if_exists_strategy, index=False, chunksize=10000)

print(f"\n✓ Data saved successfully!")


SAVING DATA TO SQLITE DATABASE

✓ Database Connection:
  Database: data/risk_profiling.db
  Table: risk_profiling_monthly_data

✓ Saving data to database...
  Strategy: replace
  Records to insert: 125,000

✓ Data saved successfully!


## 6. Verify Database

In [7]:
print(f"\n{'='*100}")
print(f"DATABASE VERIFICATION")
print(f"{'='*100}")

with engine.connect() as conn:
    # Count total records
    result = conn.execute(text(f"SELECT COUNT(*) FROM {TABLE_NAME}"))
    total_records = result.fetchone()[0]
    
    # Count records for this snapshot
    result = conn.execute(text(f"SELECT COUNT(*) FROM {TABLE_NAME} WHERE snapshot_date = '{SNAPSHOT_DATE}'"))
    snapshot_records = result.fetchone()[0]
    
    # Get all snapshot dates
    result = conn.execute(text(f"SELECT DISTINCT snapshot_date FROM {TABLE_NAME} ORDER BY snapshot_date DESC"))
    snapshots = [row[0] for row in result.fetchall()]
    
    # Get column names
    result = conn.execute(text(f"SELECT * FROM {TABLE_NAME} LIMIT 1"))
    columns = list(result.keys())
    
    # Count by data_split for current snapshot
    result = conn.execute(text(f"""
        SELECT data_split, COUNT(*) 
        FROM {TABLE_NAME} 
        WHERE snapshot_date = '{SNAPSHOT_DATE}'
        GROUP BY data_split
    """))
    split_counts = {row[0]: row[1] for row in result.fetchall()}

print(f"\n✓ Database Information:")
print(f"  Database: {DB_PATH}")
print(f"  Table: {TABLE_NAME}")
print(f"  Total Records (all snapshots): {total_records:,}")
print(f"  Records for snapshot {SNAPSHOT_DATE}: {snapshot_records:,}")
print(f"  Total Columns: {len(columns)}")

print(f"\n✓ All Snapshot Dates in Database:")
for i, snap in enumerate(snapshots, 1):
    with engine.connect() as conn:
        result = conn.execute(text(f"SELECT COUNT(*) FROM {TABLE_NAME} WHERE snapshot_date = '{snap}'"))
        count = result.fetchone()[0]
    print(f"  {i}. {snap}: {count:,} records")

print(f"\n✓ Data Split Distribution (Snapshot: {SNAPSHOT_DATE}):")
for split, count in split_counts.items():
    print(f"  {split:12s}: {count:>6,} records ({count/snapshot_records*100:.1f}%)")

print(f"\n✓ Column Names ({len(columns)}):")
print(f"  First 10: {columns[:10]}")
print(f"  Last 5:   {columns[-5:]}")

# Get database file size
db_size_mb = os.path.getsize(DB_PATH) / 1024**2
print(f"\n✓ Database File Size: {db_size_mb:.2f} MB")

print(f"\n{'='*100}")
print(f"✓ DATA LOADING COMPLETE!")
print(f"{'='*100}")
print(f"\n✓ Summary:")
print(f"  Loaded {snapshot_records:,} records for snapshot {SNAPSHOT_DATE}")
print(f"  Database now contains {total_records:,} total records across {len(snapshots)} snapshot(s)")
print(f"  Database file: {DB_PATH}")
print(f"\n✓ You can now run the main ML pipeline notebook!")


DATABASE VERIFICATION

✓ Database Information:
  Database: data/risk_profiling.db
  Table: risk_profiling_monthly_data
  Total Records (all snapshots): 125,000
  Records for snapshot 2025-11-30: 125,000
  Total Columns: 112

✓ All Snapshot Dates in Database:
  1. 2025-11-30: 125,000 records

✓ Data Split Distribution (Snapshot: 2025-11-30):
  Test        : 18,750 records (15.0%)
  Train       : 87,500 records (70.0%)
  Validation  : 18,750 records (15.0%)

✓ Column Names (112):
  First 10: ['customer_id', 'customer_segment', 'age', 'gender', 'marital_status', 'dependents', 'education', 'city_tier', 'employment_status', 'occupation_sector']
  Last 5:   ['retirement_planning_score', 'tax_planning_efficiency', 'data_split', 'risk_profile', 'snapshot_date']

✓ Database File Size: 122.38 MB

✓ DATA LOADING COMPLETE!

✓ Summary:
  Loaded 125,000 records for snapshot 2025-11-30
  Database now contains 125,000 total records across 1 snapshot(s)
  Database file: data/risk_profiling.db

✓ You c

## 7. Sample Query Test

In [8]:
# Test loading data back from database
print(f"\n{'='*100}")
print(f"SAMPLE QUERY TEST")
print(f"{'='*100}")

query = f"SELECT * FROM {TABLE_NAME} WHERE snapshot_date = '{SNAPSHOT_DATE}' LIMIT 5"
print(f"\n✓ Query: {query}")

sample_df = pd.read_sql(query, engine)
print(f"\n✓ Sample Data from Database:")
display(sample_df)

print(f"\n✓ Data types:")
print(sample_df.dtypes)


SAMPLE QUERY TEST

✓ Query: SELECT * FROM risk_profiling_monthly_data WHERE snapshot_date = '2025-11-30' LIMIT 5

✓ Sample Data from Database:


,customer_id,customer_segment,age,gender,marital_status,dependents,education,city_tier,employment_status,occupation_sector,...,insurance_adequacy_ratio,credit_card_payment_history_score,loan_repayment_track_record,financial_planning_score,budgeting_discipline_score,retirement_planning_score,tax_planning_efficiency,data_split,risk_profile,snapshot_date
0,100000,Mass Market,50,Male,Single,4,Graduate,Metro,Salaried,Other,...,23.63,54.3,14.1,80.2,7.5,98.7,77.2,Train,Conservative,2025-11-30
1,100001,Mass Market,32,Male,Single,2,Professional,Metro,Salaried,IT,...,18.22,32.3,51.9,70.3,36.4,97.2,96.2,Train,Aggressive,2025-11-30
2,100002,Emerging Affluent,58,Male,Married,0,Others,Tier 1,Salaried,Finance,...,14.29,0.9,10.1,66.4,0.5,16.1,54.9,Train,Aggressive,2025-11-30
3,100003,Emerging Affluent,62,Male,Married,4,Graduate,Tier 1,Business Owner,Government,...,27.68,79.8,65.0,70.2,79.6,89.0,33.8,Test,Aggressive,2025-11-30
4,100004,Mass Market,44,Female,Married,1,Post Graduate,Metro,Self-Employed,IT,...,15.57,4.8,94.9,88.7,26.1,1.5,93.3,Train,Aggressive,2025-11-30



✓ Data types:
customer_id                    int64
customer_segment              object
age                            int64
gender                        object
marital_status                object
                              ...   
retirement_planning_score    float64
tax_planning_efficiency      float64
data_split                    object
risk_profile                  object
snapshot_date                 object
Length: 112, dtype: object


## 8. Database Maintenance (Optional)

In [9]:
# Optional: Create indexes for faster queries
print(f"\n{'='*100}")
print(f"DATABASE OPTIMIZATION (OPTIONAL)")
print(f"{'='*100}")

print(f"\n✓ Creating indexes for faster queries...")

with engine.connect() as conn:
    # Create index on snapshot_date
    try:
        conn.execute(text(f"CREATE INDEX IF NOT EXISTS idx_snapshot_date ON {TABLE_NAME}(snapshot_date)"))
        print(f"  ✓ Created index: idx_snapshot_date")
    except Exception as e:
        print(f"  ℹ️  Index idx_snapshot_date already exists")
    
    # Create index on data_split
    try:
        conn.execute(text(f"CREATE INDEX IF NOT EXISTS idx_data_split ON {TABLE_NAME}(data_split)"))
        print(f"  ✓ Created index: idx_data_split")
    except Exception as e:
        print(f"  ℹ️  Index idx_data_split already exists")
    
    # Create composite index
    try:
        conn.execute(text(f"CREATE INDEX IF NOT EXISTS idx_snapshot_split ON {TABLE_NAME}(snapshot_date, data_split)"))
        print(f"  ✓ Created index: idx_snapshot_split")
    except Exception as e:
        print(f"  ℹ️  Index idx_snapshot_split already exists")
    
    conn.commit()

print(f"\n✓ Database optimization complete!")

# Optional: Vacuum database to reclaim space
print(f"\n✓ Running VACUUM to optimize database file...")
with engine.connect() as conn:
    conn.execute(text("VACUUM"))
    conn.commit()

new_db_size_mb = os.path.getsize(DB_PATH) / 1024**2
print(f"  Database size after VACUUM: {new_db_size_mb:.2f} MB")
print(f"  Space saved: {db_size_mb - new_db_size_mb:.2f} MB")

print(f"\n✓ All optimizations complete!")


DATABASE OPTIMIZATION (OPTIONAL)

✓ Creating indexes for faster queries...
  ✓ Created index: idx_snapshot_date
  ✓ Created index: idx_data_split
  ✓ Created index: idx_snapshot_split

✓ Database optimization complete!

✓ Running VACUUM to optimize database file...
  Database size after VACUUM: 129.40 MB
  Space saved: -7.02 MB

✓ All optimizations complete!
